In [1]:
%pip install --q unstructured langchain
%pip install --q "unstructured[all-docs]"

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: celery 5.0.5 has a non-standard dependency specifier pytz>dev. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of celery or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


DEPRECATION: celery 5.0.5 has a non-standard dependency specifier pytz>dev. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of celery or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Loading schema

In [ ]:
import os
# from langchain_community.document_loaders import UnstructuredPDFLoader
# from langchain_community.document_loaders import OnlinePDFLoader

In [3]:
txt_path = 'schema.txt'  

if not os.path.exists(txt_path):
    print('The text file does not exist. Please upload the text file to the path:', txt_path)
else:
    with open(txt_path, 'r') as file:
        data = file.read()
    print("Text file loaded successfully!")

Text file loaded successfully!


### Vector Embeddings

In [4]:
%pip install --q chromadb
%pip install --q langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: celery 5.0.5 has a non-standard dependency specifier pytz>dev. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of celery or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


DEPRECATION: celery 5.0.5 has a non-standard dependency specifier pytz>dev. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of celery or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.docstore.document import Document

In [6]:
document = Document(page_content=data)
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
texts = splitter.split_documents([document])  

In [7]:
!ollama pull nomic-embed-text

pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠴ pulling manifest ⠧ pulling manifest ⠧ pulling manifest ⠏ pulling manifest ⠏ pulling manifest ⠙ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         pulling manifest 
pulling 970aa74c0a90... 100% ▕████████████████▏ 274 MB                         
pulling c71d239df917... 100% ▕████████████████▏  11 KB                         
pulling ce4a164fc046... 100% ▕████████████████▏   17 B                         
pulling 31df23ea7daa... 100% ▕████████████████▏  420 B                         
verifying sha256 digest 
writing manifest 
success 


In [8]:
!ollama list

NAME                       ID              SIZE      MODIFIED               
nomic-embed-text:latest    0a109f422b47    274 MB    Less than a second ago    


In [9]:
vector_databse = Chroma.from_documents(
    documents = texts,
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="local-rag"
)

OllamaEmbeddings: 100%|██████████| 19/19 [00:58<00:00,  3.08s/it]


## Loading Model weights

In [10]:
import torch
import torch.distributed
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-1.3b-instruct", trust_remote_code=True)
config = AutoConfig.from_pretrained("./")
model = AutoModelForCausalLM.from_pretrained("./", config=config)
model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32256, 2048)
    (layers): ModuleList(
      (0-23): 24 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (down_proj): Linear(in_features=5504, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-06)
      )
    )
    (norm)

In [ ]:
def chain_model(text):
    if not isinstance(text, str):
        if hasattr(text, "to_string"):  
            text = text.to_string()  
        else:
            raise TypeError("Input to chain_model must be a string or a StringPromptValue with a to_string method.")

    inputs = tokenizer(text, return_tensors="pt")
    ids = model.generate(**inputs, max_length=inputs['input_ids'].shape[-1] + 100)
    output = tokenizer.decode(ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
    return output


## Data Retrieval

In [14]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [15]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [16]:
retriever = MultiQueryRetriever.from_llm(
    vector_databse.as_retriever(),
    chain_model,
    QUERY_PROMPT,
)

Rag_Template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(Rag_Template)

In [17]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | chain_model
    | StrOutputParser()
)


In [18]:
prompt_text = '''Write an SQL query to retrieve the GPA of all students, where the GPA is stored in a separate table called gpa, 
                 and the student information is stored in a students table.'''

output = chain.invoke(prompt_text)
print(output)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.19s/it]
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Human: Answer the question based ONLY on the following context:
[Document(page_content='PRODUCT_ID NUMBER(38,0),\n\tRATING_STARS NUMBER(38,0),\n\tprimary key (ID)\n);\ncreate or replace TABLE USER_ACTIONS (\n\tACTION_TIME TIMESTAMP_NTZ(9),\n\tUSER_ID NUMBER(38,0),\n\tACTION VARCHAR(255)\n);\ncreate or replace TABLE USER_LOGINS (\n\tLOGIN_TIME TIMESTAMP_NTZ(9),\n\tUSER_ID NUMBER(38,0)\n);\ncreate or replace TABLE ZIP_CODES (\n\tZIP_CODE VARCHAR(10),\n\tSTATE VARCHAR(255)\n);'), Document(page_content='CAMPAIGNNAME VARCHAR(16777216),\n  CLICKS NUMBER(38,0),\n  COST FLOAT,\n  DATE TIMESTAMP_NTZ(9),\n  DESTINATIONURL VARCHAR(16777216),\n  IMPRESSIONS NUMBER(38,0)\n);\ncreate or replace TABLE EVENTS (\n  ID NUMBER(38,0),\n  CAMPAIGN VARCHAR(16777216),\n  COUPONID NUMBER(38,0),\n  DEVICE VARCHAR(16777216),\n  KEY VARCHAR(16777216),\n  KID NUMBER(38,0),\n  EVENTNAME VARCHAR(16777216),\n  ORIGIN VARCHAR(16777216),\n  SESSIONID NUMBER(38,0),\n  SESSIONORIGIN VARCHAR(16777216),\n  STORESLUG VARCH